In [1]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sys

Using TensorFlow backend.


<br>
<br>

# 데이터 로드

In [2]:
# 텍스트 로드
text = open('./dataset/novel/novel.txt').read().lower()

In [3]:
# 텍스트 출력
text


'여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시림에 관한 책에서 기막힌 그림 하나를 본 적이 있다. 맹수를 집어삼키고 있는 보아 구렁이 그림이었다. 위의 그림은 그것을 옮겨 그린 것이다.\n 그 책에는 이렇게 씌어 있었다.\n "보아 구렁이는 먹이를 씹지도 않고 통째로 집어삼킨다.그리고는 꼼짝도 하지 못하고 여섯 달 동안 잠을 자면서 그것을 소화시킨다."\n 나는 그래서 밀림 속에서의 모험에 대해 한참 생각해 보고 난 끝에 색연필을 가지고 내 나름대로 내 생애 첫번째 그림을 그려보았다. 나의 그림 제 1호였다. 그것은 이런 그림이었다.\n 나는 그 걸작품을 어른들에게 보여 주면서내 그림이 무섭지 않느냐고 물었다.\n 그들은 "모자가 뭐가 무섭다는 거니?" 하고 대답했다.\n 내 그림은 모자를 그린 게 아니었다. 그것은 코끼리를 소화시키고 있는 보아 구렁이었다.\n 그래서 나는 어른들이 알아볼 수 있도록 보아 구렁이의 속을 그렸다. 어른들은 언제나 설명을 해주어야만 한다. 나의 그림 제 2호는 이러했다.\n 어른들은 속이 보이거나 보이지 않거나 하는 보아 구렁이의 그림들은 집어치우고 차라리 지리, 역사, 계산, 그리고 문법 쪽에 관심을 가져보는 게 좋을 것이라고 충고해 주었다.\n 그래서 나는 여섯 살 적에 화가라는 멋진 직업을 포기해 버렸다.내 그림제 1호와 제 2호가 성공을 거두지 못한 데 낙심해 버렸던 것이다. 어른들은언제나 스스로는 아무것도 이해하지 못한다.자꾸자꾸 설명을 해주어야 하니 맥빠지는 노릇이 아닐 수 없다.\n 그래서 다른 직업을 선택하지 않을 수 없게 된 나는 비행기 조종하는 법을배웠다.세계의 여기저기 거의 안 가본 데 없이 나는 날아다녔다.그러니지리는 정말로 많은 도움을 준 셈이었다.한번 슬쩍 보고도 중국과 애리조나를 나는 구별할 수 있었던 것이다.그것은 밤에 길을 잃었을 때 아주 유용한 일이다.\n 나는 그리하여 일생 동안 수없이 많은 점잖은 사람들과수많은 접촉을 가져왔다.어른들 틈에서 많이 살아온 것이다.나는 가까이서 그들을 볼

In [4]:
# 전체 글자수
len(text)


45772

<br>
<br>

# 전처리

In [5]:
# 입력으로 들어가는 글자수
input_len = 30

# 입력 문장 리스트
input_sentence_list = []

# 출력 캐릭터 리스트
output_char_list = []

# 입력 문장과 바로 다음에 나오는 출력 캐릭터 데이터셋 생성
for i in range(0, len(text) - input_len):
    input_sentence_list.append(text[i: i + input_len])
    output_char_list.append(text[i + input_len])

로드한 텍스트 문서로 입력과 출력의 데이터셋을 만듭니다. 30글자씩 입력으로 하고 바로 그 다음 글자를 출력으로 합니다. 전체 텍스트에서 오른쪽으로 하나씩 이동하면서 계속 데이터를 생성합니다. 이렇게 스스로 라벨을 만드는 머신러닝 방식을 자기지도학습(self-supervised learning)이라고 합니다.
<br>
<br>
<br>

In [6]:
# 입력 문장과 출력 캐릭터 출력
for i in range(0, 10):
    print(input_sentence_list[i] + ' -> ' + output_char_list[i])
    print()


여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시 -> 림

섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시림 -> 에

 살 적에 나는 "체험한 이야기"라는 제목의, 원시림에 ->  

살 적에 나는 "체험한 이야기"라는 제목의, 원시림에  -> 관

 적에 나는 "체험한 이야기"라는 제목의, 원시림에 관 -> 한

적에 나는 "체험한 이야기"라는 제목의, 원시림에 관한 ->  

에 나는 "체험한 이야기"라는 제목의, 원시림에 관한  -> 책

 나는 "체험한 이야기"라는 제목의, 원시림에 관한 책 -> 에

나는 "체험한 이야기"라는 제목의, 원시림에 관한 책에 -> 서

는 "체험한 이야기"라는 제목의, 원시림에 관한 책에서 ->  



In [7]:
# 입력 문장 리스트 개수
len(input_sentence_list)


45742

In [8]:
# 캐릭터 글자 추출
chars = sorted(list(set(text)))
chars


['\n',
 ' ',
 '!',
 '"',
 '(',
 ')',
 ',',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '<',
 '>',
 '?',
 'b',
 '使',
 '儀',
 '商',
 '大',
 '小',
 '布',
 '式',
 '律',
 '惑',
 '星',
 '的',
 '紀',
 '紅',
 '詩',
 '隊',
 '가',
 '각',
 '간',
 '갈',
 '감',
 '갑',
 '값',
 '갔',
 '강',
 '갖',
 '같',
 '개',
 '객',
 '거',
 '걱',
 '건',
 '걷',
 '걸',
 '검',
 '겁',
 '것',
 '게',
 '겐',
 '겠',
 '겨',
 '겪',
 '견',
 '결',
 '겸',
 '겹',
 '겼',
 '경',
 '곁',
 '계',
 '고',
 '곡',
 '곤',
 '곧',
 '골',
 '곰',
 '곱',
 '곳',
 '공',
 '과',
 '관',
 '광',
 '괜',
 '괴',
 '굉',
 '교',
 '구',
 '국',
 '군',
 '굴',
 '굽',
 '궁',
 '궂',
 '권',
 '귀',
 '귄',
 '규',
 '그',
 '극',
 '근',
 '글',
 '금',
 '급',
 '긋',
 '긍',
 '기',
 '긴',
 '길',
 '깊',
 '까',
 '깎',
 '깐',
 '깔',
 '깜',
 '깨',
 '깼',
 '꺼',
 '꺾',
 '껍',
 '껏',
 '껐',
 '께',
 '껴',
 '꼬',
 '꼭',
 '꼼',
 '꼿',
 '꽃',
 '꽝',
 '꽤',
 '꾀',
 '꾸',
 '꾼',
 '꿀',
 '꿈',
 '뀌',
 '끄',
 '끈',
 '끌',
 '끝',
 '끼',
 '낀',
 '낄',
 '낌',
 '나',
 '낙',
 '난',
 '날',
 '낡',
 '남',
 '납',
 '났',
 '낭',
 '낮',
 '내',
 '낼',
 '냅',
 '냈',
 '냐',
 '냥',
 '너',
 '넌',
 '널',
 '넓',
 '넘',
 '넣

In [9]:
# 전체 캐릭터 개수
len(chars)


851

In [10]:
# 캐릭터와 인덱스로 짝지어진 딕셔너리 생성
char_dict = dict((char, chars.index(char)) for char in chars)
char_dict


{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '3': 11,
 '4': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 '<': 18,
 '>': 19,
 '?': 20,
 'b': 21,
 '使': 22,
 '儀': 23,
 '商': 24,
 '大': 25,
 '小': 26,
 '布': 27,
 '式': 28,
 '律': 29,
 '惑': 30,
 '星': 31,
 '的': 32,
 '紀': 33,
 '紅': 34,
 '詩': 35,
 '隊': 36,
 '가': 37,
 '각': 38,
 '간': 39,
 '갈': 40,
 '감': 41,
 '갑': 42,
 '값': 43,
 '갔': 44,
 '강': 45,
 '갖': 46,
 '같': 47,
 '개': 48,
 '객': 49,
 '거': 50,
 '걱': 51,
 '건': 52,
 '걷': 53,
 '걸': 54,
 '검': 55,
 '겁': 56,
 '것': 57,
 '게': 58,
 '겐': 59,
 '겠': 60,
 '겨': 61,
 '겪': 62,
 '견': 63,
 '결': 64,
 '겸': 65,
 '겹': 66,
 '겼': 67,
 '경': 68,
 '곁': 69,
 '계': 70,
 '고': 71,
 '곡': 72,
 '곤': 73,
 '곧': 74,
 '골': 75,
 '곰': 76,
 '곱': 77,
 '곳': 78,
 '공': 79,
 '과': 80,
 '관': 81,
 '광': 82,
 '괜': 83,
 '괴': 84,
 '굉': 85,
 '교': 86,
 '구': 87,
 '국': 88,
 '군': 89,
 '굴': 90,
 '굽': 91,
 '궁': 92,
 '궂': 93,
 '권': 94,
 '귀': 95,
 '귄': 96,
 '규': 97,
 '그': 98,
 '극': 99,
 '근': 100

텍스트를 처리하는 방법에는 단어 기반과 캐릭터 기반이 있습니다. 여기서는 한 글자 단위로 하는 캐릭터 방식으로 하겠습니다. 전체 문서에서 고유한 글자 집합을 딕셔너리로 생성합니다. 여기서 만든 인덱스 번호를 다시 원핫인코딩(one-hot encoding)으로 변환합니다. 이렇게 글자를 신경망이 처리할 수 있는 숫자 벡터로 만들어야 합니다.
<br>
<br>
<br>

In [11]:
# 문장을 원핫인코딩으로 설정하기 위해 초기화
x_train = np.zeros((len(input_sentence_list), input_len, len(chars)), dtype=np.int8)
y_train = np.zeros((len(input_sentence_list), len(chars)), dtype=np.int8)

# 문장에서 각 캐릭터 위치에 1로 설정하고 나머지는 모두 0
for i, sentence in enumerate(input_sentence_list):
    for j, char in enumerate(sentence):
        x_train[i, j, char_dict[char]] = 1
        
    y_train[i, char_dict[output_char_list[i]]] = 1

이전 IMDb 영화 리뷰 분류에서는 한 문장을 하나의 벡터에 모두 표현하는 BoW(Bag-of-Words)를 사용했습니다. 이번에는 하나의 문장을 각 글자로 나누고 이를 원핫인코딩으로 변환하도록 하겠습니다. 예를 들면, 다음과 같습니다.

사 : 010<br>
랑 : 100<br>
해 : 001<br>

사랑해
-> (010, 100, 001)
<br>
<br>
<br>

In [12]:
# 훈련셋 출력
x_train


array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [13]:
# 훈련셋 라벨 출력
y_train


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

<br>
<br>

# 모델 생성

In [14]:
def build_model():
    
    model = models.Sequential()
    model.add(layers.LSTM(128, input_shape=(input_len, len(chars))))
    model.add(layers.Dense(len(chars), activation='softmax'))
    
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy')
    
    return model    

input_shape에서 input_len는 입력의 글자 수로 순환신경망의 타임 스텝입니다. 중요한 점은 신경망의 입력이 'input_len x len(chars)'의 크기가 아니라는 것입니다. 입력 노드의 개수는 len(chars)이고, 이 하나의 신경망 모델을 input_len의 타임 스텝만큼 반복하여 돌립니다.

<br>
<br>

# 훈련 및 테스트

In [15]:
# 다음 캐릭터 글자를 선택
def select_char(logits, temperature=1.0):
    
    # 모델에서 각 글자가 나올 확률값인 로짓을 온도에 따라 확률을 분산시킴
    # 온도가 낮으면 확률이 조금, 온도가 높으면 확률이 많이 분산됨
    logits = np.asarray(logits).astype('float64')
    logits = np.log(logits) / temperature
    exp_logits = np.exp(logits)
    logits = exp_logits / np.sum(exp_logits)
    
    # 다항분포로 랜덤하게 로짓의 특정 위치를 선택
    # random.multinomial(반복횟수, 확률값, 크기)
    # 확률값에 따라서 딱 한번 주사위를 돌림
    # 로짓의 특정 위치가 1이되고 나머지는 0으로 설정됨
    results = np.random.multinomial(1, logits, 1)
    
    # 캐릭터 글자의 인덱스 구함
    index = np.argmax(results) 
    
    return index

logits는 softmax로 나온 신경망의 출력입니다. 글자의 개수만큼 차원이 존재하고 각각 그 글자가 나올 확률값이 들어있습니다. 텍스트를 생성할 때 약간 변화를 주기 위해서 온도라는 개념을 적용합니다. 온도가 낮으면 확률값에 변화가 거의 없습니다. 반면에 온도가 높으면 확률값을 조금씩 주변으로 분배합니다. 예를 들어, 다음과 같습니다.

< 처음 logits ><br>
가 -> 0.1<br>
나 -> 0.7<br>
다 -> 0.2<br>

< 온도 0.2 ><br>
가 -> 0.13<br>
나 -> 0.64<br>
다 -> 0.23<br>

< 온도 1.0 ><br>
가 -> 0.2<br>
나 -> 0.5<br>
다 -> 0.3<br>

이렇게 확률값을 온도에 따라 변화시켜도 바로 argmax를 적용하면 처음과 동일한 글자가 선택됩니다. 그래서 random.multinomial()을 사용하여 수정된 확률에 따라 글자를 결정하도록 합니다.

< 다항분포 적용 ><br>
가 -> 0<br>
나 -> 1 (0.5의 확률로 선택됨)<br>
다 -> 0
<br>
<br>
<br>

In [16]:
# 시작 문장이 주어지면 이어지는 텍스트 생성
def generate_text(model, start_text):
        
    print('\n- 시작 텍스트 :' + start_text)
    print()

    # 온도를 변화시키며 텍스트 생성
    for temperature in [0.2, 0.5, 1.0]:
        print('- 온도 :', temperature)
        
        # 시작 텍스트로 텍스트 생성 초기화
        generated_text = start_text
        sys.stdout.write(generated_text)

        # 시작 텍스트에 이어지는 200개의 글자를 생성
        for i in range(200):
            # 텍스트 벡터 초기화
            text_vector = np.zeros((1, input_len, len(chars)))
            
            # 텍스트를 원핫인코딩으로 변경
            for j, char in enumerate(generated_text):
                text_vector[0, j, char_dict[char]] = 1.

            # 다음 글자를 온도에 따라 확률을 변경하여 결정
            logits = model.predict(text_vector, verbose=0)
            char_index = select_char(logits[0], temperature)
            char_next = chars[char_index]
            
            # 다음 글자를 추가
            generated_text += char_next
            
            # 첫 번째 글자를 삭제하여 전체 입력 길이 유지
            generated_text = generated_text[1:]

            # 캐릭터 글자 출력
            sys.stdout.write(char_next)
            sys.stdout.flush()
            
        print('\n')    

시작 문장이 주어지면 거기에 이어지는 200개의 글자를 생성합니다. 입력 문장의 바로 다음 글자를 예측하고, 이 글자가 다시 입력 문장에 추가됩니다. 동시에 처음 글자는 삭제되어 전체 입력 문장의 길이를 유지합니다.

동해물과백두산 -> 이<br>
해물과백두산이 -> 마<br>
물과백두산이마 -> 르
<br>
<br>
<br>

In [17]:
# 랜덤 시드 선택
random.seed(1212)

# 시드 텍스트의 위치를 랜덤하게 결정
start_index = random.randint(0, len(text) - input_len - 1)

# 모델 생성
model = build_model()

# 훈련 반복
for epoch in range(0, 50):
    print('Total Epoch :', epoch + 1)
    
    # 모델 훈련
    model.fit(x_train,
              y_train,
              epochs=1,
              batch_size=128)

    # 텍스트 생성
    start_text = text[start_index: start_index + input_len]
    generate_text(model, start_text)


Total Epoch : 1
Epoch 1/1
45742/45742 [==============================] - 150s 3ms/step - loss: 4.4010

- 시작 텍스트 :?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지.

- 온도 : 0.2
?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지.          이  는  다      이        이    이         이     이 다   이   이        다   이  이    이      이  다................... "   이       는   다   이      이  다      이   이    다          다              이     을  다   

- 온도 : 0.5
?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지.  위  어에 
  모 는  .  다...
 그"  은 했는 지.   이      다.." 이..... .......
 그나  은  는  다  다   아   을  명는서 업  이 아 "   라 하  가   다   한 단 시  이고 했다.... "" 어 다  그  ?..
...  " 는  리이 어는  억 하  시  없 를  부다   이 이   이   에 이 

- 온도 : 1.0
?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지.  이"었...
?".가 건 요했 들의 에산구었에런 겠와았다러 그그 라했  들는 여저이
를 나는다래 은나 이그는의  야각 에 생른을 능다 붉구 깨중 조 다가)..에러하꽃스근은는에하 그할아다이도 
리 이는 말에 만서구에이있..
이 의어있디 즉왕 어하는 잔없을 더왕사야 근란만도 있겠린들 볼다자을 에이면내할  람 은 가는대고주에 럼을 씨우린인다은 것다을 밀!에에

Total Epoch : 2
Epoch 1/1
45742/45742 [==============================] - 144s 3ms/step - loss: 3.8004

- 시작 텍스트 :?"
 어린 왕자가 그에

 "아저씨도 모래 별들 모두 아저씨게 잘 못한 거든. 그들은 소유하게 되

- 온도 : 1.0
?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지."
 슨 말했다.
 "아, 사람들이 아무도 하늘 생을 사막어. 사람들을 위에서 2거르렸다! 여우우는 그래서 섯 너심방스로 쑤로터째 그림을 호져 있었다. 만들은 무없이었다.나무는 잘못노제다. 말만 아저씨" 하지만 꽃 모든."
 그우 돌란느 리에서 노라. 제호게 생각하다. 나는 만외들을 때문에게 몹우 원저했과 말사인 가오없는 여러분은 다우 신사의 을것하고 오래

Total Epoch : 20
Epoch 1/1
45742/45742 [==============================] - 143s 3ms/step - loss: 2.0811

- 시작 텍스트 :?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지.

- 온도 : 0.2
?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지."
 어린 왕자가 물었다.
 "그래. 아저씨 가로등 켜는 사람들 사람 이 있는 것이다. 그것은 그것은 너무 무엇이 어린 아무도 없는 것이었다.
 "그들은 아무것도 하지 않는 것이다."
 "그래. 그래. 그래. 그래서 나는 그 별들을 아주 해 해야 하는 거야. 그들은 아저씨 사람들이 아저씨 것은 그들이 아 있는 것이다. 그 꽃은 어른들은 모두 두 상상에 대해

- 온도 : 0.5
?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지."
 여우가 말했다.
 "그래 너는 어른이 아니라. 건 사람들이 아무 거야!"
 "아니, 그들이 말했다.
 "아저씨들이 아무것 것이 네 가 되는 거야."
 "그래. 내가 이었다.
 "아저씨는 장미꽃 이 마리 사람에 조금 보이지 않는 것이다. 그들이 그 소리를 그리 하지 않았다. 그것은 모든 것들을 아주 게 했다. 그것은 그를 소리를 알아니라고 있다는 우로 

- 온도 : 1.0
?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지."
 어린 왕자가 물었다.
 그처럼 렇지. 하빛면서 모청를 계무해 주려 명령

 "사람들은 이건 아무것도 모래서 노라이 별들이 아저씨에게 한 웃 만만 어진 그들은 모두 가장 사 위람에 가로등 켜는 이가 있었던 것이었다. 그러나 그는 별들에 중대한 일이 있는 사람이에게 나는 이제 이제 이야기에서 나를 생각해 보다는 것을 나고 말고 그것이다.
 나는 그 꽃은 그림을 바라보았다. 그러면 그런 데도 그의 말을 바라보았다

- 온도 : 0.5
?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지."
 여우가 말했다.
 "기자를 조금만 다시고 말했다.그 수도 있었다. 그가 말했다.
 그는 말을 가할 수 있었다. 그러나 나는 그런데 그것은 그의 말을 어리는 것을 하지도 않았다.
 "
 그렇지? 그런데 잘 짐이 어린 왕자는 게만 하지 않을 위해 해지 만한 것이다. 그것은 아주 두랑세러 나는 보고 무대고 있는 것이지.그러나 내가 그를 어린 아는 도만 하지

- 온도 : 1.0
?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지."
 여우가 말했다.
 어린 왕자가 릇이를 생위었다. 그 소난리를 내리네 말으로 내 얼더을 물을 떠거야. 떤조을 심뱀없어.나는 이지나에게 마음을운래서 양이이추었어세에게는 모른소는 된 바있다. 가장운 사람에도 비리가 고장 하다 의의 그림을 있다니지 않는 만를 보깨구나 그는 보고 모난 따음로 심포할 수 있었다.내가 그 꽃에서 매조번 복그 한5일 여기 법이를감에

Total Epoch : 39
Epoch 1/1
45742/45742 [==============================] - 207s 5ms/step - loss: 1.3398

- 시작 텍스트 :?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지.

- 온도 : 0.2
?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지."
 여우가 말했다.
 "아저씨 다. 그들 말이다."
 "그래."
 "그 꽃은 "
 "그건지 모르게 아무것을 하게 수있는 어른들 이 나는 이거가지를 하고 있는 것도 모습이 있어.."
 그리고는 그 꽃을 이었다.
 "아저씨에게 나는 것들을 여러분이

In [18]:
# 모델 저장
model.save('./model/text_generation_model.h5')

<br>
<br>

# 텍스트 생성

In [19]:
# 저장된 모델 파일 로드
model = models.load_model('./model/text_generation_model.h5')

In [20]:
# 새로운 문장으로 텍스트 생성
start_text = '사막이 아름다운 것은 그것이 어딘가에 샘을 감추고 있기 때문이지'

if len(start_text) > input_len:
    start_text = start_text[:input_len]

if len(start_text) < input_len:
    start_text = start_text + ' '*(input_len - len(start_text))

generate_text(model, start_text)   



- 시작 텍스트 :사막이 아름다운 것은 그것이 어딘가에 샘을 감추고 있기

- 온도 : 0.2
사막이 아름다운 것은 그것이 어딘가에 샘을 감추고 있기 때문이지....."
 그가 그 꽃은 어린 왕자는 생각이 있었다.
 "다른 별들을 모두 게 보이거나 아저구나를 위해 보다. 그는 소리를 지아니라 말야.나는 한 가로등 을 전의 사막을 있었다.
 "사람들은 어떤 모래의처럼 사실에게도 마나도,래도 곳다. 아의 사람들이 사막하고 여러분은 사람을 고장을 수 있다.그건 나를 생각이 들었어.어린 그는 중요한 일이 어른

- 온도 : 0.5
사막이 아름다운 것은 그것이 어딘가에 샘을 감추고 있기 때문이지....."
 그리 그는 이렇게 말하고 이 세상에 대답나 말이 되었지.어린 왕자는 무래서 디도 그를 웃었다.
 "하게 해야 겠어. 그래서 그럼 별들이는소유하고 있다는 거야?"
 어린 왕자가 대해 하지 않았다. 그는 것을 다른 들여기 아주었다. 그런 다는 별을서 살고 있고 그 마음을 나는 것이다. 어린아왕이는 다시 별이 있었어.그들은 서장한 조어진 

- 온도 : 1.0
사막이 아름다운 것은 그것이 어딘가에 샘을 감추고 있기다집가운2곤한 나의 조금 그진 금빛이 다지 붉있는고는 어린 왕자이 말했다.
 "제를 휘거야. 나석에게 존태면워천혀을 좋을 나면..... 그래도 그러나 뭘 하나 보일 어떤 직르들을 처럼 도시한 지고 기고도도 물었다.
 그러
 어린 왕자가 오를 심시렸다.할 별의 아름서지리도 않고 있굴에 발심했이 때길었다......
 그리고언히그 말고 풍의 생각로 생 것다다워웃



온도가 낮으면 어느정도 그럴듯한 문장이 생성되지만, 훈련 텍스트에 있는 문장만 나오기가 쉽습니다. 반면에 온도가 높으면 다양한 문장이 만들어지지만, 약간 문법에 맞지 않는 경우가 생길 수 있습니다.

파라미터의 크기가 작고 데이터가 적기 때문에 문장의 품질이 그리 좋지 않습니다. GPT2같은 모델은 진짜 사람이 쓴 것과 구별이 힘들 정도로 뛰어난 텍스트를 생성합니다.